In [ ]:
import requests, gzip
from dicttoxml2 import dicttoxml
from pathlib import Path

def scrape_animelist(user):
	# Get the user's anime list
	url = f"https://myanimelist.net/animelist/{user}/load.json?status=7"
	start_offset = 0
	page_size = 300

	result = []
	while True:
		# Get the data
		print(f"Scraping with offset {start_offset}...")
		data = requests.get(f"{url}&offset={start_offset}").json()
		result.extend(data)

		# Increment the offset
		data_length = len(data)
		start_offset += data_length

		# Check if we're done
		if data_length < page_size:
			print(f"Scraped {start_offset} entries, done!")
			break

	return result

def convert_to_exported_format(scrapped_data):
	# Initialize the result with some default values
	result = {
	}

	conversion_map = {
		'anime_id': 'series_animedb_id',
		'anime_title': 'series_title',
		'anime_num_episodes': 'series_episodes',
		'anime_media_type_string': 'series_type',
		'score': 'my_score',
		'notes': 'my_comments',
		'num_watched_episodes': 'my_watched_episodes',
		'finish_date_string': 'my_finish_date',
		'start_date_string': 'my_start_date',
		'tags': 'my_tags',
		'is_rewatching': 'my_rewatching',
		'storage_string': 'my_storage',
		'priority_string': 'my_priority',
		'status': 'my_status',
	}

	# Convert the data using the map
	for key_scrapped, key_export in conversion_map.items():
		result[key_export] = scrapped_data.get(key_scrapped, None)

	status_map = {
		1: 'Watching',
		2: 'Completed',
		3: 'On Hold',
		4: 'Dropped',
		6: 'Plan to Watch',
	}

	# Convert the status
	result['my_status'] = status_map.get(result['my_status'], None)

	return result

# Get username from data/user.txt
data = Path('data')
user = (data / 'user.txt').read_text().strip()

# Scrapes the anime list of the user
animelist = scrape_animelist(user)

# Convert to MyAnimeList XML export format
animelist = [convert_to_exported_format(data) for data in animelist]
xml = dicttoxml(
	{'anime': animelist},
	custom_root='myanimelist',
	attr_type=False,
	cdata=True,
	fold_list=False,
)
print(xml)

output = data / 'animelist.xml.gz'
print(f"Saving animelist to {output}...")
with gzip.open(output, 'w') as f:
	f.write(xml)